# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor
warnings.filterwarnings('ignore')

### Изучение данных

In [ ]:
data = pd.read_csv("/datasets/autos.csv")
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [ ]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


None

**Признаки**  
DateCrawled — дата скачивания анкеты из базы  
VehicleType — тип автомобильного кузова  
RegistrationYear — год регистрации автомобиля  
Gearbox — тип коробки передач  
Power — мощность (л. с.)  
Model — модель автомобиля  
Kilometer — пробег (км)  
RegistrationMonth — месяц регистрации автомобиля  
FuelType — тип топлива  
Brand — марка автомобиля  
NotRepaired — была машина в ремонте или нет  
DateCreated — дата создания анкеты  
NumberOfPictures — количество фотографий автомобиля  
PostalCode — почтовый индекс владельца анкеты (пользователя)  
LastSeen — дата последней активности пользователя  
**Целевой признак**  
Price — цена (евро)  

In [ ]:
data[data.duplicated(keep=False)]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
41529,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15
88087,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
90964,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
171088,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
187735,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
231258,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
258109,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
325651,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15


In [ ]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


**Важные признаки**: VehicleType, Gearbox, Power, Kilometer, FuelType, Brand, NotRepaired, RegistrationYear, Model.  

Признаки NumberOfPictures, PostalCode не несут полезной информации, их можно удалить.  
DateCrawled, LastSeen, NumberOfPictures могли быть полезными для прогнозирования скорости продажи но для прогнозирования цены авто они не нужны.  
DateCreated может быть полезна для анализа с учетом инфляции.

По многим важным параметрам есть пропуски и нулевые значеня необходимо будет восстановить их или удалить.  
Есть неадекватные значения параметров необходимо будет восстановить их или удалить  
Найдено небольшое количество дубликатов необходимо их удалить

### Предобработка данных

Обраотаем целевой признак price, пропусков данных нет, но есть цена равная нулю, восстановить по среднему было бы некоректно, так как признак целевой и это напрямую повлияет на прогноз.

In [ ]:
print("Колличество объявлений с нулевой ценой:",len(data.loc[data['Price'] == 0]))

Колличество объявлений с нулевой ценой: 10772


In [ ]:
data = data.loc[data['Price'] != 0]

Имеются пропуски в Model к сожалению их нельзя восстановит по Brand придется удалить

In [ ]:
print("Колличество объявлений с пропущенной моделью:", len(data.loc[data['Model'].isna()]))

Колличество объявлений с пропущенной моделью: 17521


In [ ]:
data = data.loc[~data['Model'].isna()]

Колличество пропусков в VehicleType около 10 процентов, данных слишком много чтобы просто от них избавится, если заменить на среднюю это тоже может сказаться на точности предсказания. Если брать в расчет что в дальнейшем пользователи при оценке автомобиля могут так же не вводить тип кузова, то стоит заменить пропущенные значения на unknown

In [ ]:
print("Колличество объявлений с незаполненным типом кузова:", len(data.loc[data['VehicleType'].isna()]))

Колличество объявлений с незаполненным типом кузова: 28166


In [ ]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')

Количество явных некорректных данных RegistrationYear незначительно, можно их удалить

In [ ]:
print("Колличество объявлений с некорректной годом:",len(data.loc[(data['RegistrationYear'] > 2021) | (data['RegistrationYear'] < 1769)]))

Колличество объявлений с некорректной годом: 83


In [ ]:
data = data.loc[(data['RegistrationYear'] <= 2021) & (data['RegistrationYear'] >= 1900)]

Колличество пропусков в Gearbox тоже велико. Заменю на наиболее встречающийся тип коробки в модели.  

In [ ]:
print("Колличество объявлений с незаполненным типом коробки:", len(data.loc[data['Gearbox'].isna()]))


Колличество объявлений с незаполненным типом коробки: 14205


In [ ]:

data['Gearbox'] = data['Gearbox'].fillna(data
                                         .groupby('Model')['Gearbox']
                                         .transform(lambda x: x.value_counts().idxmax())
                                        )

Имеются значения Power равные 0 и больше 1000 что являеться некорректным, можно заменить их на медиану по модели

In [ ]:
print("Колличество объявлений с некорректной мощностью:",len(data.loc[(data['Power'] > 1000) | (data['Power'] <= 0)]))

Колличество объявлений с некорректной мощностью: 31519


In [ ]:
data.loc[(data['Power'] > 1000) | (data['Power'] <= 0), 'Power'] = None
data['Power'] = data['Power'].fillna(data.groupby('Model')['Power'].transform('median'))
data = data.loc[~data['Power'].isna()]
data['Power'] = data['Power'].astype('int64')

Пропуски в FuelType заменю на среднее по моделям

In [ ]:
data['FuelType'] = data['FuelType'].fillna(data.groupby('Model')['FuelType'].transform(lambda x: x.value_counts().idxmax()))

Пропуски в NotRepaired состоявляют треть от данных. Скорее всего если автомобиль не был поврежден то этот параметр просто не заполнялся. Заменю на "yes"

In [ ]:
data["NotRepaired"].fillna("yes", inplace=True)

Признак DateCreated преобразуем в количество дней с момента 2014-03-01.

In [ ]:
print("Минимальная дата создания объявления:", min(data['DateCreated']))

Минимальная дата создания объявления: 2014-03-10 00:00:00


In [ ]:
base_date = pd.Timestamp(min(data['DateCreated']))
data['DateCreated'] = data['DateCreated'].map(lambda date : (pd.Timestamp(date) - base_date).days)

Удалим дубликаты

In [ ]:
data = data.drop_duplicates()

Удалю ненужные признаки NumberOfPictures, PostalCode, DateCrawled, LastSeen, 

In [ ]:
data = data.drop(['NumberOfPictures', 'PostalCode', 'DateCrawled', 'LastSeen'], axis=1)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325985 entries, 0 to 325984
Data columns (total 12 columns):
Price                325985 non-null int64
VehicleType          325985 non-null object
RegistrationYear     325985 non-null int64
Gearbox              325985 non-null object
Power                325985 non-null int64
Model                325985 non-null object
Kilometer            325985 non-null int64
RegistrationMonth    325985 non-null int64
FuelType             325985 non-null object
Brand                325985 non-null object
NotRepaired          325985 non-null object
DateCreated          325985 non-null int64
dtypes: int64(6), object(6)
memory usage: 29.8+ MB


In [ ]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,DateCreated
count,325985.000000,325985.000000,325985.000000,325985.000000,325985.000000,325985.000000
mean,4623.374517,2003.268147,119.117377,128597.067350,5.853966,741.798098
std,4530.487896,7.061790,53.172506,36963.747615,3.665989,9.399255
min,1.000000,1910.000000,1.000000,5000.000000,0.000000,0.000000
25%,1200.000000,1999.000000,75.000000,125000.000000,3.000000,734.000000
50%,2950.000000,2003.000000,110.000000,150000.000000,6.000000,742.000000
75%,6650.000000,2008.000000,145.000000,150000.000000,9.000000,750.000000
max,20000.000000,2019.000000,1000.000000,150000.000000,12.000000,759.000000


### Кодирование категориальных признаков

Преобразую категориальные признаки с большим колличеством категорий техникой Ordinal Encoding.

In [ ]:
data_reg = data

In [ ]:
encoder = OrdinalEncoder()
encoder.fit(data) 
data_ordinal = pd.DataFrame(encoder.transform(data), columns=data.columns)

data_ordinal.reset_index()
data.loc[:,['Brand', 'Model']] = data_ordinal.loc[:,['Brand', 'Model']]
data['Brand'] = data['Brand'].astype(int)
data['Model'] = data['Model'].astype(int)

Признаки VehicleType, Gearbox, FuelType, NotRepaired закодируем one-hot encoding.

In [ ]:
def dum(data, column):
    df = pd.get_dummies(data[column], prefix=column, drop_first=True)
    data = data.drop(column, axis=1)
    return data.join(df)

In [ ]:
data = dum(data, 'VehicleType')
data = dum(data, 'Gearbox')
data = dum(data, 'FuelType')
data = dum(data, 'NotRepaired')

Разобью данные на тестовую и валидационную выборки и на признаки и целевой признак

In [ ]:
target = data['Price']
features = data.drop('Price', axis=1)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)



In [ ]:
features_train

,RegistrationYear,Power,Model,Kilometer,RegistrationMonth,Brand,DateCreated,VehicleType_convertible,VehicleType_coupe,VehicleType_other,...,VehicleType_unknown,VehicleType_wagon,Gearbox_manual,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol,NotRepaired_yes
236576,2004,163,80,150000,4,21,732,0,1,0,...,0,0,1,0,0,0,0,0,1,1
11305,2006,435,31,150000,11,1,729,0,0,0,...,0,1,0,0,0,0,0,0,1,0
158561,2005,150,28,150000,11,1,754,0,1,0,...,0,0,0,0,0,0,0,0,1,0
90386,2000,60,166,125000,3,5,731,0,0,0,...,0,0,1,0,0,0,0,0,1,0
81972,1999,50,143,150000,7,37,742,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158838,2005,224,31,150000,4,1,742,0,0,0,...,0,1,0,0,1,0,0,0,0,0
47873,1997,320,206,150000,8,20,730,1,0,0,...,0,0,0,0,0,0,0,0,1,0
86398,2003,142,11,150000,6,2,749,0,0,0,...,0,0,1,0,0,0,0,0,1,1
77285,1994,143,59,5000,10,20,737,0,0,0,...,0,0,1,0,0,0,0,0,1,0


Стандартизирую численные признаки и преобразованные техникой Ordinal Encoding.

In [ ]:
numeric = ['RegistrationYear', 'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'Brand',  'DateCreated']

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric]) 
print(features_train.head())

        RegistrationYear     Power     Model  Kilometer  RegistrationMonth  \
236576          0.102262  0.822256 -0.410821   0.580456          -0.506371   
11305           0.385380  5.921662 -1.106259   0.580456           1.402882   
158561          0.243821  0.578534 -1.148836   0.580456           1.402882   
90386          -0.463974 -1.108769  0.809743  -0.095333          -0.779122   
81972          -0.605533 -1.296248  0.483313   0.580456           0.311880   

           Brand  DateCreated  VehicleType_convertible  VehicleType_coupe  \
236576  0.036914    -1.044481                        0                  1   
11305  -1.503673    -1.363732                        0                  0   
158561 -1.503673     1.296699                        0                  1   
90386  -1.195556    -1.150898                        0                  0   
81972   1.269385     0.019692                        0                  0   

        VehicleType_other  ...  VehicleType_unknown  VehicleType_wag

**Вывод**  
Дубликаты удалены.
Отобраны признаки, необходимые для построения моделей.  
Пропущенные значения, выбросы, неправдоподобные значения в признаках заполнены на основе имеющейся инфрмации или удалены.  
Категориальные признаки преобразованы с помощью one-hot encoding.   
Признак с большим колличеством категорий закодирован техникой Ordinal Encoding.  
Данные разделены на тестовую и обучающую выборки и стандартизированны  

## Анализ моделей

###  Linear Regression

In [ ]:
%%time
lmodel = LinearRegression()
lmodel.fit(features_train, target_train)

CPU times: user 310 ms, sys: 178 ms, total: 489 ms
Wall time: 476 ms


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
%%time
preds_lmodel = lmodel.predict(features_test)

CPU times: user 24.7 ms, sys: 8.36 ms, total: 33 ms
Wall time: 81.5 ms


In [ ]:
mse_lmodel = mean_squared_error(target_test, preds_lmodel)
print("RMSE для линейной модели на тестовой выборке:", round((mse_lmodel) ** 0.5, 2))

RMSE для линейной модели на тестовой выборке: 2892.27


###  Регрессия Ridge

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time

rmodel = Ridge()
hyperparams = [{'solver':['auto', 'svd', 'cholesky', 'lsqr','sparse_cg']}]
rmodel = GridSearchCV(rmodel, hyperparams, scoring='neg_mean_squared_error')
rmodel.fit(features_train, target_train)
print(rmodel.best_params_)

{'solver': 'auto'}
CPU times: user 5.41 s, sys: 3.39 s, total: 8.8 s
Wall time: 8.79 s


In [ ]:
%%time
preds_rmodel = rmodel.predict(features_test)

CPU times: user 11.9 ms, sys: 12.5 ms, total: 24.3 ms
Wall time: 86.9 ms


In [ ]:
mse_rmodel = mean_squared_error(target_test, preds_rmodel)
print("RMSE для Ridge модели на тестовой выборке:", round((mse_rmodel) ** 0.5, 2))

RMSE для Ridge модели на тестовой выборке: 2892.26


### Регрессия DecisionTreeRegresso

In [ ]:
%%time

trmodel = DecisionTreeRegressor(criterion='mse', 
                              max_depth=8, 
                              random_state=12345) 
trmodel.fit(features_train, target_train)

CPU times: user 835 ms, sys: 9.08 ms, total: 844 ms
Wall time: 854 ms


DecisionTreeRegressor(criterion='mse', max_depth=8, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=12345, splitter='best')

In [ ]:
%%time
preds_trmodel = trmodel.predict(features_test)

CPU times: user 17.2 ms, sys: 0 ns, total: 17.2 ms
Wall time: 15.6 ms


In [ ]:
mse_trmodel = mean_squared_error(target_test, preds_trmodel)
print("RMSE для DecisionTreeRegresso модели на тестовой выборке:", round((mse_trmodel) ** 0.5, 2))

RMSE для DecisionTreeRegresso модели на тестовой выборке: 2120.34


### Регрессия CatBoostRegressor

In [ ]:
numerical_features = ['DateCreated', 'Price', 'RegistrationYear', 'Power', 'Kilometer']


In [ ]:
categorical_features = [col for col in list(features_train.columns) if col not in numerical_features]

связи с тем, что CatBoostRegressor принемает только целые числа или строки в категориях, пришлость пересобрать признаки.  
    Так же попробывал оставить категориальные признаки без one-hot encoding, качество немного выросло на CatBoost и LGBM, но время обработки многократно увеличилось



In [ ]:
data_reg.loc[:,['VehicleType','Gearbox', 'Brand', 'FuelType', 'NotRepaired']] = data_ordinal.loc[:,['VehicleType','Gearbox', 'Brand', 'FuelType', 'NotRepaired']]
catfeatures = data_reg.drop('Price', axis=1)
cattarget = data_reg['Price']
catfeatures_train, catfeatures_test, cattarget_train, cattarget_test = train_test_split(features, target, test_size=0.25, random_state=12345)


In [ ]:
numericcat = ['RegistrationYear', 'Power', 'Kilometer', 'DateCreated']
scaler = StandardScaler()
scaler.fit(catfeatures_train[numericcat])
catfeatures_train[numericcat] = scaler.transform(catfeatures_train[numericcat])
catfeatures_test[numericcat] = scaler.transform(catfeatures_test[numericcat]) 

In [ ]:
%%time
catmodel = CatBoostRegressor(learning_rate=0.5, random_state=12345, verbose=False) 
catmodel.fit(catfeatures_train, target_train, cat_features=categorical_features)

CPU times: user 8min 38s, sys: 1min 5s, total: 9min 43s  
Wall time: 9min 46s

In [ ]:
%%time

cat_predict = catmodel.predict(catfeatures_test)

CPU times: user 5.22 s, sys: 45.4 ms, total: 5.27 s  
Wall time: 5.22 s

In [ ]:
mse_catmodel = mean_squared_error(target_test, cat_predict)
print("RMSE для CatBoostRegressor модели на тестовой выборке:", round((mse_catmodel) ** 0.5, 2))

RMSE для CatBoostRegressor модели на тестовой выборке: 1624.89

### Регрессия LGBMRegressor

In [ ]:
%%time

lgbmmodel = LGBMRegressor(learning_rate=0.1, 
                      num_leaves=100, 
                      random_state=12345,
                      cat_features=categorical_features)
lgbmmodel.fit(features_train, target_train)

CPU times: user 12.4 s, sys: 72.8 ms, total: 12.5 s
Wall time: 12.6 s


LGBMRegressor(boosting_type='gbdt',
              cat_features=['Model', 'RegistrationMonth', 'Brand',
                            'VehicleType_convertible', 'VehicleType_coupe',
                            'VehicleType_other', 'VehicleType_sedan',
                            'VehicleType_small', 'VehicleType_suv',
                            'VehicleType_unknown', 'VehicleType_wagon',
                            'Gearbox_manual', 'FuelType_electric',
                            'FuelType_gasoline', 'FuelType_hybrid',
                            'FuelType_lpg', 'FuelType...,
                            'NotRepaired_yes'],
              class_weight=None, colsample_bytree=1.0, importance_type='split',
              learning_rate=0.1, max_depth=-1, min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=100, objective=None, random_state=12345,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=

In [ ]:
%%time

lgbm_predict = lgbmmodel.predict(features_test)

CPU times: user 1.41 s, sys: 0 ns, total: 1.41 s
Wall time: 1.41 s


In [ ]:
mse_lgbmmodel = mean_squared_error(target_test, lgbm_predict)
print("RMSE для LGBMRegressor модели на тестовой выборке:", round((mse_lgbmmodel) ** 0.5, 2))

RMSE для LGBMRegressor модели на тестовой выборке: 1664.7


Проведены измерения:  
 - времени обучения  
 - времени предсказания моделей  
 - качества предсказания по метрике RMSE

## Анализ моделей

In [ ]:
df = [["495 ms", "50 ms", 2898.79],
        ["916 ms", "63.4 ms", 2898.78],
        ["892 ms", "16.9 ms", 2118.02],
        ["9min 46s", "5.22 s", 1624.89],
        ["12.3 s", "1.41 s", 1651.87]]
model = ["Linear Regression", "Ridge", "DecisionTreeRegresso", "CatBoostRegressor", "LGBMRegressor"]

In [ ]:
pd.DataFrame(data=df, index=model, columns=["fit_time", "predict_time", "RMSE"])

,fit_time,predict_time,RMSE
Linear Regression,495 ms,50 ms,2898.79
Ridge,916 ms,63.4 ms,2898.78
DecisionTreeRegresso,892 ms,16.9 ms,2118.02
CatBoostRegressor,9min 46s,5.22 s,1624.89
LGBMRegressor,12.3 s,1.41 s,1651.87


**Вывод**:  
Наилучшее качество получилось на модели Catboost, время обучения было дольше остальных и составило 9 min 46s.  
LGBMRegressor предсказала незначительно хуже, но обучилась заметно быстрее на данных гиперпараметрах 12.2 s,